<a href="https://colab.research.google.com/github/munj1/gan-test/blob/main/test_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/test"
!kaggle datasets download -d jessicali9530/celeba-dataset

100% 1.33G/1.33G [00:59<00:00, 23.8MB/s]
100% 1.33G/1.33G [00:59<00:00, 23.8MB/s]


In [6]:
!unzip -q celeba-dataset.zip -d .